### 라이브러리

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [2]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [3]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [4]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [5]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [6]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [21]:
train.groupby('월')['중식계'].mean().sort_values()

월
12    827.336634
11    847.333333
7     848.736364
8     854.785047
6     863.311111
5     869.787879
4     883.621359
9     907.148936
10    917.771739
1     934.247525
3     953.607477
2     989.623656
Name: 중식계, dtype: float64

In [7]:
month1 = {
    1:3,
    2:1,
    3:2,
    4:6,
    5:7,
    6:8,
    7:10,
    8:9,
    9:5,
    10:4,
    11:11,
    12:12
}

month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [8]:
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [9]:
train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1
train['공휴일전후'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1

In [10]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후'])
test = pd.get_dummies(test, columns=['공휴일전후'])


In [11]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

In [12]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

# 현재 1월만 대체
train['석식계'][244] = 478.8605851979346

#train['석식계'][281] = 398
train['석식계'][492] = 478.8605851979346

#train['석식계'][502] = 398
#train['석식계'][510] = 398
#train['석식계'][529] = 398
train['석식계'][730] = 478.8605851979346

#train['석식계'][747] = 398
#train['석식계'][766] = 398
train['석식계'][973] = 478.8605851979346

#train['석식계'][993] = 398

# 1 2 3 월 자기개발의날 대체.

In [24]:
# 원핫인코딩 # 중식요일!
train = pd.get_dummies(train, columns=['요일'])
test = pd.get_dummies(test, columns=['요일'])


In [25]:
train

,일자,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,요일(석식),년,월,일,주,식사가능자수,월(중식),월(석식),공휴일전후_0,공휴일전후_1,공휴일전후_2,요일_0,요일_1,요일_2,요일_3,요일_4
0,2016-02-01,2601,50,150,238,0.0,1039.0,331.0,1,2016,2,1,5,2551.0,1,2,1,0,0,1,0,0,0,0
1,2016-02-02,2601,50,173,319,0.0,867.0,560.0,2,2016,2,2,5,2551.0,1,2,1,0,0,0,1,0,0,0
2,2016-02-03,2601,56,180,111,0.0,1017.0,573.0,4,2016,2,3,5,2545.0,1,2,1,0,0,0,0,1,0,0
3,2016-02-04,2601,104,220,355,0.0,978.0,525.0,3,2016,2,4,5,2497.0,1,2,1,0,0,0,0,0,1,0
4,2016-02-05,2601,278,181,34,0.0,925.0,320.0,5,2016,2,5,5,2323.0,1,2,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,2983,75,198,4,391.0,1093.0,421.0,4,2021,1,20,3,2517.0,3,11,1,0,0,0,0,1,0,0
1201,2021-01-21,2983,92,231,462,351.0,832.0,353.0,3,2021,1,21,3,2540.0,3,11,1,0,0,0,0,0,1,0
1202,2021-01-22,2983,255,248,1,303.0,579.0,207.0,5,2021,1,22,3,2425.0,3,11,1,0,0,0,0,0,0,1
1203,2021-01-25,2983,107,153,616,327.0,1145.0,502.0,1,2021,1,25,4,2549.0,3,11,1,0,0,1,0,0,0,0


### 최종제출 ( 자기개발1대체,금)

In [13]:
train.loc[(train.요일==4), '석식계'] -= 10

In [14]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [15]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]

In [16]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 203.7657572	total: 172ms	remaining: 2m 51s
1:	learn: 198.9612742	total: 175ms	remaining: 1m 27s
2:	learn: 193.9685729	total: 177ms	remaining: 58.7s
3:	learn: 189.5320403	total: 178ms	remaining: 44.4s
4:	learn: 185.3923703	total: 187ms	remaining: 37.1s
5:	learn: 181.1885504	total: 189ms	remaining: 31.4s
6:	learn: 177.1277591	total: 191ms	remaining: 27.1s
7:	learn: 173.1696087	total: 194ms	remaining: 24.1s
8:	learn: 169.4192587	total: 196ms	remaining: 21.6s
9:	learn: 165.9011097	total: 199ms	remaining: 19.7s
10:	learn: 162.5156921	total: 201ms	remaining: 18.1s
11:	learn: 159.2746261	total: 204ms	remaining: 16.8s
12:	learn: 156.5375684	total: 206ms	remaining: 15.6s
13:	learn: 154.1137930	total: 208ms	remaining: 14.7s
14:	learn: 151.2739116	total: 210ms	remaining: 13.8s
15:	learn: 148.6534175	total: 212ms	remaining: 13.1s
16:	learn: 146.2654245	total: 214ms	remaining: 12.4s
17:	learn: 144.0488649	total: 216ms	remaining: 11.8s
18:	learn: 141.9260794	t

In [17]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,1014.917278,321.318684
1,2021-01-28,980.706284,507.473362
2,2021-01-29,575.655728,233.021222


In [18]:
submission

,일자,중식계,석식계
0,2021-01-27,1014.917278,321.318684
1,2021-01-28,980.706284,507.473362
2,2021-01-29,575.655728,233.021222
3,2021-02-01,1229.619436,518.737171
4,2021-02-02,1049.321770,569.793711
5,2021-02-03,1061.483756,515.775621
6,2021-02-04,1006.667667,565.792804
7,2021-02-05,699.966797,369.800979
8,2021-02-08,1293.014016,683.592530
9,2021-02-09,1067.772512,585.701251


In [115]:
submission['석식계'].mean()

521.7725867393168

In [19]:
submission.to_csv('자기개발1대체,금다운중식.csv', index=False)

In [22]:
train.groupby('요일')['중식계'].mean()

요일
0    1146.395833
1     925.620833
2     905.213389
3     823.991803
4     653.590909
Name: 중식계, dtype: float64